In [1]:
from energy.energy_functions import *
from algos.SA import simulated_annealing

solutions =[]
algo_energy = []
solution_energy = []
pg_energy = []

solution_cost = []
pg_cost = []

solution_latency = []
pg_latency = []


query_names = ['3b','1a','32a','8a','7a','25a','19a','22a','24a','28a','29b']


for query in query_names:
    print("----------------------------------------------------------")
    print(query)
    with open('/home/said/Desktop/projects/job_algos/JOB-queries/'+ str(query) + '.sql', 'r') as file:

        query = file.read()
        psensor = findPowerSensor("YWATTMK1-1F6860.power")
        stopDataRecording(psensor)
        clearPowerMeterCache(psensor)
        tm = time.time()
        datalog = psensor.get_dataLogger()
        datalog.set_timeUTC(time.time())
        startDataRecording(psensor)  # Power Meter starts recording power per second
        time.sleep(2.0)
        print("4 - is recording: ", psensor.get_dataLogger().get_recording())
        startTime= datetime.fromtimestamp( time.time()).strftime('%Y-%m-%d %H:%M:%S')

        #apply the algorithm
        solution , cost = simulated_annealing(query,10, 8 ,0.5)

        endTime= datetime.fromtimestamp( time.time()).strftime('%Y-%m-%d %H:%M:%S')
        # print("4-4 - is recording: ", psensor.get_dataLogger().get_recording())
        print("startTime: ", startTime, " - endTime: ", endTime)
        # YAPI.Sleep(2000)
        time.sleep(2.0)
        print("stop recording : ", datetime.now())
        stopDataRecording(psensor)
        print("7 - is recording: ", psensor.get_dataLogger().get_recording())

        (power, exec_time, energy) = getAveragePower(psensor, startTime, endTime)

        (query_power, query_exec_time, query_energy) = get_query_exec_energy(input, False)

        (query_power, query_exec_time, solution_query_energy) = get_query_exec_energy(solution, True)

        solution_cost.append(cost)
        algo_energy.append(energy)
        solution_energy.append(solution_query_energy)
        pg_energy.append(query_energy)
        solutions.append(solution)

    print("solution_cost : ", solution_cost)
    print("algo_energy :", algo_energy)
    print("solution_energy", solution_energy)
    print("pg_energy", pg_energy)


----------------------------------------------------------
3b
4 - is recording:  1
mother loop index 0
best cost 34228.3
mother loop index 1
best cost 34228.3
mother loop index 2
best cost 34228.3
File 3b.sql created in directory solutions/sa
startTime:  2023-04-12 20:58:53:782212  - endTime:  2023-04-12 20:58:54:023977
stop recording :  2023-04-12 20:58:56.026082
7 - is recording:  0
avg_power :  119.076
Time(s):  0.241765  - AVG Power(w):  119.076  - Energy(J):  28.78840914
4 - is recording:  1
startTime:  2023-04-12 20:58:58:859  - endTime:  2023-04-12 20:59:40:005248
stop recording :  2023-04-12 20:59:42.007429
7 - is recording:  0
avg_power :  121.13502325581395
Time(s):  41.146248  - AVG Power(w):  121.13502325581395  - Energy(J):  4984.251708369488
4 - is recording:  1
startTime:  2023-04-12 20:59:44:847  - endTime:  2023-04-12 20:59:45:388849
stop recording :  2023-04-12 20:59:47.390117
7 - is recording:  0
avg_power :  118.859
Time(s):  0.541849  - AVG Power(w):  118.859  - En